In [4]:
##### multiclass.py from SF-Fire-Risk project

import pandas as pd


def XY_data(multiclass=False):
    #will process binary or multiclass

    k=pd.read_csv('data/molly_master_11152017.csv',low_memory=False)
    # set target to Fire Incident Type
    y=k.pop('Incident_Dummy')

    # assign classes
    # Nan becomes no incident
    # Values either become an incident or classes of incidents
    #y=y.apply(lambda x:'0 No incident' if pd.isnull(x) else x if multiclass else '1 Incident')

    #store class labels
    #unique=sorted(y.unique())

    #substitue class index number for class description
    #y=y.apply(lambda x:unique.index(x))

    # set x to remaining data
    x=k
    #calculate property age
    x['age']=2016-x.Yr_Property_Built
    #create one-hot variables for property type and neighborhood

    return x,y #,unique

def Data_normalized(multiclass=False):

    x,y = XY_data()

    ### ?? What is property_code_des?? 

    x_dummies=pd.get_dummies(data=x[['Neighborhood', 'rur_urb']],drop_first=True)

    # get quantitative features
    x_quantitative=x[['Num_Stories','Num_Units', 'Perc_Ownership', 'Land_Value', 
        'Property_Area','Assessed_Improvement_Val', 'Tot_Rooms', 'landval_psqft',
        'count_potential_fire_control', 'count_all_complaints',  'population', 
        'median_age', 'avg_household_size', 'num_housing_units', 'occupied', 
        'vacant', 'seasonal', 'gini_a', 'gini_b', 'num_under_18']]

    #normalize quantitative features
    x_scaled=(x_quantitative-x_quantitative.mean())/(x_quantitative.max()-x_quantitative.min())
    # i'll try ^ this first .. but then go back and normalize by population


    #combine x dummies and x scaled data
    x_all=pd.concat([x_dummies,x_scaled],axis=1)
    return x_all,y  #,unique


def classifier(train=True,x=None,y=None,class_weight=None,multiclass=False,plot=False,cross_val=False):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.multiclass import OneVsRestClassifier

    # use multiclass random forest classifier for both binary and multiclass
    if multiclass:

        rf_model=OneVsRestClassifier(RandomForestClassifier(verbose=0,class_weight=class_weight),n_jobs=3)
    else:
        rf_model = RandomForestClassifier(verbose=0, class_weight=class_weight)

    xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.33)

    import pickle
    train=train

    if train: # run training and pickle model else just load model
        rf_model.fit(xtrain,ytrain)
        # output file name
        output=open('clf.pkl','wb')
        s = pickle.dump(rf_model,output)
    # load output file
    load=open('clf.pkl','rb')
    # return model
    rf_model = pickle.load(load)

    print('training accuracy {:.2f}'.format(rf_model.score(xtrain,ytrain)))

    print('testing accuracy {:.2f}'.format(rf_model.score(xtest,ytest)))

    ypred=rf_model.predict(xtest)
    ypred=pd.DataFrame(ypred)

    from sklearn.metrics import classification_report
    #print('labels {}'.format(target_names))
    ytest=ytest.reset_index(drop=True)

    print(classification_report(ytest,ypred))
    #print(multiclass)


    if multiclass == False:
        from sklearn.metrics import roc_curve

        fpr, tpr, thresh=roc_curve(ytest,rf_model.predict_proba(xtest)[:,1])

        import matplotlib.pyplot as plt
        import numpy as np
        plt.plot(fpr,tpr,linestyle='-')
        plt.plot([0,1],[0,1],linestyle='--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve for Binary Class')
        if plot:
            plt.show()
    print([xtrain.columns[i] for i in np.argsort(rf_model.feature_importances_)[::-1]])

    from sklearn.model_selection import cross_val_score
    if cross_val:
        scores=cross_val_score(rf_model,X=x,y=y,cv=10)
        print('cross validation {}'.format(scores))



if __name__ == '__main__':

    x,y = Data_normalized(multiclass=False)
    #
    #classifier(train=True,x=x,y=y,target_names=target_names, class_weight=None,multiclass=False,plot=False,cross_val=True)
    classifier(train=True,x=x,y=y, class_weight=None,multiclass=False,plot=False,cross_val=True)

training accuracy 0.98
testing accuracy 0.93
             precision    recall  f1-score   support

        0.0       0.94      0.98      0.96     56144
        1.0       0.78      0.58      0.67      8128

avg / total       0.92      0.93      0.92     64272

['Property_Area', 'Assessed_Improvement_Val', 'Land_Value', 'landval_psqft', 'Tot_Rooms', 'count_potential_fire_control', 'avg_household_size', 'Num_Stories', 'Num_Units', 'median_age', 'population', 'num_under_18', 'num_housing_units', 'gini_a', 'occupied', 'vacant', 'gini_b', 'Perc_Ownership', 'count_all_complaints', 'seasonal', 'Neighborhood_SUNSET/PARKSIDE', 'Neighborhood_MISSION', 'Neighborhood_OUTER RICHMOND', 'Neighborhood_INNER SUNSET', 'Neighborhood_WEST OF TWIN PEAKS', 'Neighborhood_MARINA', 'Neighborhood_TENDERLOIN', 'Neighborhood_RUSSIAN HILL', 'Neighborhood_SOUTH OF MARKET', 'Neighborhood_PACIFIC HEIGHTS', 'Neighborhood_NOB HILL', 'Neighborhood_EXCELSIOR', 'Neighborhood_HAIGHT ASHBURY', 'Neighborhood_FINANCIAL DISTRIC

In [3]:
from column_report import get_column_report

get_column_report(x)

,Column Name,Data Type,Unique Count,Sample Value,NaNs/Zeros,% NaN
54,seasonal,float64,49,-0.01,0,0.00
38,Num_Stories,float64,341,0.04,0,0.00
39,Num_Units,float64,673,-0.00,0,0.00
40,Perc_Ownership,float64,4510,0.13,0,0.00
41,Land_Value,float64,80138,-0.00,0,0.00
42,Property_Area,float64,18191,-0.00,0,0.00
43,Assessed_Improvement_Val,float64,97321,-0.00,0,0.00
53,vacant,float64,101,-0.01,0,0.00
45,landval_psqft,float64,136983,-0.04,0,0.00
44,Tot_Rooms,float64,1345,-0.01,0,0.00
